In [1]:
from numpy import log2, sqrt, log
from scipy.special import comb
from AllFunctions import *
from random import sample
import time
import matplotlib.pyplot as plt

In [2]:
class RandomForest(DecisionTree):
    def __init__(self, train, labels, attributes, depth = -1, weights = None):
        super().__init__(train, labels, attributes, depth, weights)

    
    def best_att(self, train, labels, attributes, weights = None):
    
        #k = int(sqrt(len(attributes)))
        if len(attributes)>= 2:
            k= 2
        else:
            k=1
            
        sample_attributes = sample(attributes, k)
        
        lable_Ent = entropy(labels, weights)
        Max = -1
        Best = None
        Best_values = None
    
        for attribute in sample_attributes: 
            temp, temp_values = Entropy_given_attribute(train, labels, attribute, weights) 
            if lable_Ent - temp >  Max:
                Max = lable_Ent - temp
                Best = attribute
                Best_values = temp_values
                    
        return(Best, Best_values)

In [3]:
train, train_labels, test, test_labels = load_bank()


for i in range(len(train_labels)):
    if train_labels[i] =='no':
        train_labels[i] = -1
    elif train_labels[i] =='yes':
        train_labels[i] = 1
        
    
    if test_labels[i] =='no':
        test_labels[i] = -1
    elif test_labels[i] =='yes':
        test_labels[i] = 1

# Random Forest

In [39]:
Forest = []
for x in range(200):
    X, Y = my_sample(train, train_labels)
    Forest.append (RandomForest(X, Y, [i for i in range(len(train[0]))], depth = -1))

# Bias and Variance for Forest

In [56]:
def B_predictor(List_Tress, x):
    
    count = {}
    majority = None
    Max = -1
    for j in range(len(List_Tress)):
        temp = List_Tress[j].predict(x) 
        if temp not in count:
            count[temp] = 0
                
        count[temp] += 1
        if count[temp] > Max:
            Max = count[temp]
            majority = temp
    return(majority)

In [57]:
Forest_predict = []
for i in range(len(test)):
    Forest_predict.append(B_predictor(Forest, test[i]))

In [58]:
import numpy as np
Exp_Forest = sum(Forest_predict)/5000
Var_Forest = sum((np.array(Forest_predict) - Exp_Forest)**2) / 4999
Var_Forest

0.3309421884377112

In [59]:
Exp_Gen = sum((np.array(Forest_predict) - np.array(test_labels))**2) / 5000

In [60]:
bias = Exp_Gen - Var_Forest
bias

0.19145781156228875

# Bias and Variance for a single Random Tree

In [68]:
X, Y = my_sample(train, train_labels)
A = RandomForest(X, Y, [i for i in range(len(train[0]))], depth = -1)

In [69]:
RT_predict = []
for i in range(len(test)):
    RT_predict.append(B_predictor([A], test[i]))

In [70]:
Exp_RT = sum(RT_predict)/5000
Var_RT = sum((np.array(RT_predict) - Exp_RT)**2) / 4999
Var_RT

0.45603744748946656

In [71]:
Exp_Gen_RT = sum((np.array(RT_predict) - np.array(test_labels))**2) / 5000

In [72]:
bias = Exp_Gen_RT - Var_Forest
bias

0.32665781156228874